In [1]:
import numpy as np
%matplotlib qt
import matplotlib.pyplot as pl

In [2]:
class State:
    def __init__(self,L:int,p:float,q:int) -> None:
        assert L%2 == 0
        self.p = p
        self.q = q
        self.L = L
        # index = np.arange(0,2**L,1)
        self.data = np.zeros((2,)*L)
        self.data[(0,)*L] = 1
        ##
        # no_of_ones = np.zeros(2**L,dtype=int)
        # for i in range(L):
        #     no_of_ones += temp%2
        #     temp = (temp/2).astype(int)
        # self.data = (p**2)**(L-no_of_ones)*((1 - p**2)/(q**2))**no_of_ones
        # self.data = self.data.reshape((2,)*L)

In [134]:
def eventransfer(State,U_t,U_k):
    L = State.L
    p = State.p
    q = State.q
    os = State.data.reshape((4,)*(L//2))
    for x in range(L//2):
        if x%2 == 0:
            U = U_t
        else:
            U = U_k
        os = np.tensordot(U,os,axes=(-1,x))
    os = np.reshape(os,(2,)*L)
    State.data = os


def oddtransfer(State,U_t,U_k):
    L = State.L
    p = State.p
    q = State.q
    os = State.data
    os = np.moveaxis(os,0,-1) #bringing 1st spin to the last position
    os = np.reshape(os,(4,)*(L//2))

    for x in range(L//2):
        if x%2 == 0:
            U = U_t
        else:
            U = U_k
        os = np.tensordot(U,os,axes=(-1,x))
    os = np.reshape(os,(2,)*L)
    os = np.moveaxis(os,-1,0)
    State.data = os

In [135]:
def no_of_down_spins(N):
    temp = np.arange(0,2**N,1,dtype=int)
    no_of_ones = np.zeros(2**N,dtype=float)
    for i in range(N):
        no_of_ones += temp%2
        temp = (temp/2).astype(int)
    return N - no_of_ones

def TopLayer(state,log_Z):
    """
    This function contracts the evolved state with the top layer.
    """

    ## Caluclate no. of down spins in each configuration
    no_of_downs = no_of_down_spins(state.L)
    log_Z.append((3*state.L/2)*np.log(state.q))
    fac = np.sum(state.data.reshape(2**state.L)*((state.q)**(no_of_downs-(state.L)//2)))
    log_Z.append(np.log(fac))
    return log_Z

In [136]:
def get_U_t(p,q):
    U = np.zeros((4,4))
    U[0,0] = p**2
    U[3,0] = (1-p**2)/q**2
    U[3,3] = 1
    U[0,3] = 0
    U[3,1] = q*p**2/(q**2+1) + (1-p**2)/q
    U[3,2] = q*p**2/(q**2+1) + (1-p**2)/q
    U[0,1] = p**2*q/(q**2+1)
    U[0,2] = p**2*q/(q**2+1)
    return U

def get_U_k(p,q):
    U = np.zeros((4,4))
    U[0,0] = 1
    U[3,0] = 0
    U[3,3] = p**2
    U[0,3] = (1-p**2)/q**2
    U[0,2] = q*p**2/(q**2+1) + (1-p**2)/q
    U[0,1] = q*p**2/(q**2+1) + (1-p**2)/q
    U[3,2] = p**2*q/(q**2+1)
    U[3,1] = p**2*q/(q**2+1)
    return U

In [137]:
def free_energy(L,p,q,T):

    state = State(L,p,q)
    log_Z = []
    ##
    U_t = get_U_t(p,q)
    U_k = get_U_k(p,q)
    seed=1
    rng = np.random.default_rng(seed=seed)
    
    ##

    for t in range(T):
        
        if t%2 == 0:
            eventransfer(state,U_t,U_k)
        else:
            oddtransfer(state,U_t,U_k)
        sd = np.sum(state.data)
        log_Z.append(np.log(sd))
        state.data = state.data/sd

    log_Z = TopLayer(state,log_Z)
    return log_Z,state


In [140]:
F = {}
for L in [8,10,12,14,16,18][:]:
    p_list = np.round(np.linspace(0.2,1,10),2)
    T = L
    q = 50
    F[L] = []
    for p in p_list:
        log_Z = free_energy(L,p,q,L)[0] 
        F[L].append(np.sum(log_Z)-(L*np.log(q)))

In [142]:
for L in F:
    pl.plot(p_list,np.array(F[L])/(L*np.log(q)),'-o',label = r'$L=$'+str(L))
pl.ylabel(r'$\Delta F$',fontsize=16)
pl.xlabel(r'$p$',fontsize=16)
pl.title(r'Haar random dynamics, transfer_matrix. $q=2,T=10$',fontsize=16)
#pl.legend(fontsize=16)
pl.tight_layout()

In [20]:
p_list

array([0.85, 0.87, 0.88, 0.9 , 0.92, 0.93, 0.95, 0.97, 0.98, 1.  ])

In [50]:
log_Z, state = free_energy(10,1,2,3)

In [55]:
log_Z,15*np.log(2),5*np.log(2),10*np.log(2),(np.sum(log_Z)-10*np.log(2))/(10*np.log(2))

([0.0, 0.0, 0.0, 10.39720770839918, 3.4657359027997265],
 10.39720770839918,
 3.4657359027997265,
 6.931471805599453,
 1.0)